USE THIS TO CONVERT JSON TO CSV TABLES THEN JSON TABLES

In [2]:
import pandas as pd
import json
import os

# Load JSON files into Pandas DataFrames
def load_json(file_path):
    """ Load JSON file and convert it to a Pandas DataFrame """
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return pd.DataFrame(data)

# File paths (update as per actual locations)
num_file = "D:/NEU/SEM2/BigData/ASGN2/TEST/jsons2/2009q4/num.json"
pre_file = "D:/NEU/SEM2/BigData/ASGN2/TEST/jsons2/2009q4/pre.json"
sub_file = "D:/NEU/SEM2/BigData/ASGN2/TEST/jsons2/2009q4/sub.json"
tag_file = "D:/NEU/SEM2/BigData/ASGN2/TEST/jsons2/2009q4/tag.json"

# Load data
num_df = load_json(num_file)
pre_df = load_json(pre_file)
sub_df = load_json(sub_file)
tag_df = load_json(tag_file)

# Clean NUM data
num_df["value"] = pd.to_numeric(num_df["value"], errors="coerce")  # Convert to float
num_df["ddate"] = pd.to_datetime(num_df["ddate"], format="%Y%m%d", errors="coerce")  # Convert to datetime
num_df.dropna(subset=["value"], inplace=True)  # Drop rows where value is NaN

# Clean PRE data
pre_df = pre_df[["adsh", "tag", "stmt"]]  # Keep relevant columns
pre_df.rename(columns={"stmt": "statement_type"}, inplace=True)  # Rename for clarity

# Debugging: Check if ticker column exists
print("Checking if 'ticker' column exists in sub.json...")
print("SUB Data Columns BEFORE Fix:", sub_df.columns)

# Ensure 'ticker' exists in SUB data
if "ticker" not in sub_df.columns:
    print("Warning: 'ticker' column not found in sub.json! Adding it as NaN.")
    sub_df["ticker"] = None  # Assign NaN if missing

# Debugging: Show sample data
print("SUB Data Sample AFTER Fix:\n", sub_df.head())

# Clean SUB data
required_columns = ["adsh", "cik", "filed", "fy", "fp"]
if "ticker" in sub_df.columns:
    required_columns.append("ticker")  # Include ticker only if present

sub_df = sub_df[required_columns]  # Keep only necessary columns
sub_df["filed"] = pd.to_datetime(sub_df["filed"], format="%Y%m%d", errors="coerce")  # Convert to datetime

# Clean TAG data
tag_df = tag_df[["tag", "tlabel"]]  # Keep only tag and label

# Debugging check
print("Final SUB Columns:", sub_df.columns)
print("Sample SUB Data:\n", sub_df.head())
print("NUM Sample:\n", num_df.head())
print("PRE Sample:\n", pre_df.head())
print("TAG Sample:\n", tag_df.head())


Checking if 'ticker' column exists in sub.json...
SUB Data Columns BEFORE Fix: Index(['adsh', 'cik', 'name', 'sic', 'countryba', 'stprba', 'cityba', 'zipba',
       'bas1', 'bas2', 'baph', 'countryma', 'stprma', 'cityma', 'zipma',
       'mas1', 'mas2', 'countryinc', 'stprinc', 'ein', 'former', 'changed',
       'afs', 'wksi', 'fye', 'form', 'period', 'fy', 'fp', 'filed', 'accepted',
       'prevrpt', 'detail', 'instance', 'nciks', 'aciks', 'ticker'],
      dtype='object')
SUB Data Sample AFTER Fix:
                    adsh      cik                     name   sic countryba  \
0  0001047469-09-009543    24545  MOLSON COORS BREWING CO  2082        US   
1  0001047469-09-009754   831001            CITIGROUP INC  6021        US   
2  0001157523-09-007839  1067983   BERKSHIRE HATHAWAY INC  6331        US   
3  0000918160-09-000039   918160    AK STEEL HOLDING CORP  3312        US   
4  0001144204-09-056412  1121788               GARMIN LTD  3812        KY   

  stprba        cityba     zipb

In [3]:
# Filter Balance Sheet items
bs_pre = pre_df[pre_df["statement_type"] == "BS"]

# Merge with NUM data
balance_sheet = num_df.merge(bs_pre, on=["adsh", "tag"], how="inner")

# Merge with SUB data to get filing info
balance_sheet = balance_sheet.merge(sub_df, on="adsh", how="inner")

# Merge with TAG data for descriptions
balance_sheet = balance_sheet.merge(tag_df, on="tag", how="left")

# Keep only required columns, including TICKER
balance_sheet = balance_sheet[["ticker", "cik", "filed", "fy", "fp", "tag", "tlabel", "value", "ddate"]]

print("Balance Sheet Sample:\n", balance_sheet.head())


Balance Sheet Sample:
     ticker      cik      filed    fy  fp  \
0  UNKNOWN    77476 2009-10-08  2009  Q3   
1  UNKNOWN   103379 2009-11-10  2009  Q3   
2  UNKNOWN    18230 2009-10-30  2009  Q3   
3  UNKNOWN  1326380 2009-12-09  2009  Q3   
4  UNKNOWN   915912 2009-11-09  2009  Q3   

                                                 tag  \
0  StockholdersEquityIncludingPortionAttributable...   
1  StockholdersEquityIncludingPortionAttributable...   
2  StockholdersEquityIncludingPortionAttributable...   
3                               FixturesAndEquipment   
4                 AccruedExpensesAndOtherLiabilities   

                                              tlabel         value      ddate  
0  Stockholders' Equity, Including Portion Attrib...  3.063800e+10 2008-12-31  
1  Stockholders' Equity, Including Portion Attrib...  1.108140e+08 2009-09-30  
2  Stockholders' Equity, Including Portion Attrib...  3.057000e+09 2008-12-31  
3                             Fixtures and equipment  6

In [4]:
# Filter Income Statement items
is_pre = pre_df[pre_df["statement_type"] == "IS"]

# Merge with NUM data
income_statement = num_df.merge(is_pre, on=["adsh", "tag"], how="inner")

# Merge with SUB data to get filing info
income_statement = income_statement.merge(sub_df, on="adsh", how="inner")

# Merge with TAG data for descriptions
income_statement = income_statement.merge(tag_df, on="tag", how="left")

# Keep only required columns, including TICKER
income_statement = income_statement[["ticker", "cik", "filed", "fy", "fp", "tag", "tlabel", "value", "ddate"]]

print("Income Statement Sample:\n", income_statement.head())


Income Statement Sample:
     ticker      cik      filed    fy  fp  \
0  UNKNOWN   879101 2009-11-05  2009  Q3   
1  UNKNOWN    91576 2009-11-06             
2  UNKNOWN     5272 2009-11-06  2009  Q3   
3  UNKNOWN     5272 2009-11-06  2009  Q3   
4  UNKNOWN  1385157 2009-11-18  2009  FY   

                                                 tag  \
0                                         ProfitLoss   
1               CommonStockDividendsPerShareDeclared   
2  OtherThanTemporaryImpairmentLossesInvestmentsA...   
3  OtherThanTemporaryImpairmentLossesInvestmentsA...   
4     RestructuringAndAssetImpairmentAndOtherCharges   

                                              tlabel         value      ddate  
0  Net Income (Loss), Including Portion Attributa...  2.761800e+07 2008-09-30  
1       Common Stock, Dividends, Per Share, Declared  5.625000e-01 2008-09-30  
2  Other-than-temporary Impairment Losses, Invest...  1.970600e+10 2008-09-30  
3  Other Than Temporary Impairment Losses, Invest...

In [5]:
# Filter Cash Flow items
cf_pre = pre_df[pre_df["statement_type"] == "CF"]

# Merge with NUM data
cash_flow = num_df.merge(cf_pre, on=["adsh", "tag"], how="inner")

# Merge with SUB data to get filing info
cash_flow = cash_flow.merge(sub_df, on="adsh", how="inner")

# Merge with TAG data for descriptions
cash_flow = cash_flow.merge(tag_df, on="tag", how="left")

# Keep only required columns, including TICKER
cash_flow = cash_flow[["ticker", "cik", "filed", "fy", "fp", "tag", "tlabel", "value", "ddate"]]

print("Cash Flow Sample:\n", cash_flow.head())



Cash Flow Sample:
     ticker      cik      filed    fy  fp  \
0  UNKNOWN    40545 2009-11-06  2009  Q3   
1  UNKNOWN    40545 2009-11-06  2009  Q3   
2  UNKNOWN   879101 2009-11-05  2009  Q3   
3  UNKNOWN  1403161 2009-11-20  2009  FY   
4  UNKNOWN  1403161 2009-11-20  2009  FY   

                                                 tag  \
0     EarningsFromContinuingOperationsRetainedByGECS   
1     EarningsFromContinuingOperationsRetainedByGECS   
2                                         ProfitLoss   
3  NoncashOrPartNoncashAcquisitionNonrecourseDebt...   
4  NoncashOrPartNoncashAcquisitionNonrecourseDebt...   

                                              tlabel       value      ddate  
0  Earnings from continuing operations retained b...         0.0 2008-09-30  
1  Earnings from continuing operations retained b...         0.0 2008-09-30  
2  Net Income (Loss), Including Portion Attributa...  27618000.0 2008-09-30  
3  Non-recourse debt assumed in acquisition of su...         0.0 20

In [6]:
for df_name, df in [("Balance Sheet", balance_sheet), 
                    ("Income Statement", income_statement), 
                    ("Cash Flow", cash_flow)]:
    if "tlabel" in df.columns:
        df.drop(columns=["tlabel"], inplace=True)
        print(f"Dropped 'tlabel' column from {df_name}")
    if "ticker" in df.columns:
        df.drop(columns=["ticker"], inplace=True)
        print(f"Dropped 'tlabel' column from {df_name}")

balance_sheet.to_csv("balance_sheet.csv", index=False)
income_statement.to_csv("income_statement.csv", index=False)
cash_flow.to_csv("cash_flow.csv", index=False)

print("Denormalized fact tables saved successfully!")


Dropped 'tlabel' column from Balance Sheet
Dropped 'tlabel' column from Balance Sheet
Dropped 'tlabel' column from Income Statement
Dropped 'tlabel' column from Income Statement
Dropped 'tlabel' column from Cash Flow
Dropped 'tlabel' column from Cash Flow
Denormalized fact tables saved successfully!


In [7]:
import pandas as pd

# Define file names
csv_files = {
    "balance_sheet.csv": "balance_sheet.json",
    "income_statement.csv": "income_statement.json",
    "cash_flow.csv": "cash_flow.json"
}

# Convert each CSV file to JSON
for csv_file, json_file in csv_files.items():
    try:
        df = pd.read_csv(csv_file)  # Read CSV file
        df.to_json(json_file, orient="records", indent=4)  # Save as JSON
        print(f"Converted {csv_file} -> {json_file}")
    except Exception as e:
        print(f"Error converting {csv_file}: {e}")

print("All CSV files have been successfully converted to JSON!")


Converted balance_sheet.csv -> balance_sheet.json
Converted income_statement.csv -> income_statement.json
Converted cash_flow.csv -> cash_flow.json
All CSV files have been successfully converted to JSON!


In [ ]:
# Load the large CSV file
file_path = "D:/NEU/SEM2/BigData/ASSIGNMENT2/cash_flow.csv"
df = pd.read_csv(file_path, chunksize=1000000)  # Adjust chunk size as needed

# Split into multiple smaller CSV files
for i, chunk in enumerate(df):
    chunk.to_csv(f"cash_flow_{i+1}.csv", index=False)
    print(f"Created: cash_flow_part_{i+1}.csv")

print("CSV Splitting Completed!")
